In [ ]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter 
import scipy.spatial
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

In [ ]:
#this is borrowed from https://github.com/davideverona/deep-crowd-counting_crowdnet
def gaussian_filter_density(gt):
    print (gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.gaussian_filter(pt2d, sigma, mode='constant')
    print('done.')
    return density

In [ ]:
#set the root to the Shanghai dataset you download
root = r'datesets/ShanghaiTech'

In [ ]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_B_test]

In [ ]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [ ]:
# from PIL import Image  
# import numpy as np
# import matplotlib.pyplot as plt
# from io import BytesIO

# from torchvision import datasets, transforms
# transform=transforms.Compose([
#                        transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]),
#                    ])
# model = CSRNet()
# model.eval()
# model = model.to("cpu")
# checkpoint = torch.load('partBmodel_best.pth.tar', map_location='cpu')
# model.load_state_dict(checkpoint['state_dict'])

# videoname = 'test.avi'
# capture = cv2.VideoCapture(0)
# writer=cv2.VideoWriter("hah.avi",cv2.VideoWriter_fourcc(*'MJPG'),20.0,(128,96), True)
# if capture.isOpened():
#     while True:
#         ret,img=capture.read() # img 就是一帧图片           
#         # 可以用 cv2.imshow() 查看这一帧，也可以逐帧保存
#         if not ret:break # 当获取完最后一帧就结束
#         img = img[:, :, ::-1]
#         img_pil = Image.fromarray(img)
#         img = transform(img_pil.convert('RGB')).to("cpu")
#         output = model(img.unsqueeze(0))
#         outputssss = np.squeeze(output.detach().cpu().numpy())
#         plt.imshow(outputssss,cmap=CM.jet)
#         tmpfile = BytesIO()
#         plt.savefig(tmpfile)
#         img_pil_density = Image.open(tmpfile)
#         img_np_density = np.array(Image.open(tmpfile))
#         # outputssss = cv2.resize(img_pil,(128,96))
#         # print(type(outputssss))
#         print(np.around(output.detach().cpu().sum().numpy()))
#         writer.write(img_np_density)
#         # img_pil_density.show()
#         # img_pil.show()
#         display(img_pil)
#         display(img_pil_density)
#         plt.clf()

# else:
#     print('视频打开失败！')
# writer.release()

In [ ]:
# mat = io.loadmat(r'ShanghaiTech\ShanghaiTech\part_A_final/train_data\ground_truth\GT_IMG_1.mat')
# img_path = r'ShanghaiTech\ShanghaiTech\part_A_final/train_data\images\IMG_1.jpg'
# gt = mat["image_info"][0,0][0,0][0]
# img= plt.imread(img_path)
# k = np.zeros((img.shape[0],img.shape[1]))
# for i in range(0,len(gt)):
#     if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
#         k[int(gt[i][1]),int(gt[i][0])]=1

# density = np.zeros(gt.shape, dtype=np.float32)
# gt_count = np.count_nonzero(gt)

# pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
# leafsize = 2048
# # build kdtree

# tree = scipy.spatial.KDTree(pts, leafsize=leafsize)
# # query kdtree
# distances, locations = tree.query(pts, k=4)

# print('generate density...')
# for i, pt in enumerate(pts):
#     pt2d = np.zeros(gt.shape, dtype=np.float32)
#     pt2d[pt[1],pt[0]] = 1.
#     if gt_count > 1:
#         sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
#     else:
#         sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
#     density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
# print('done.')

In [ ]:
for img_path in img_paths[0:10]:
    print(img_path)
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    print(type(k))
    k = gaussian_filter_density(k)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
            hf['density'] = k

In [ ]:
#now see a sample from ShanghaiA
plt.imshow(Image.open(img_paths[0]))

In [ ]:
gt_file = h5py.File(img_paths[0].replace('.jpg','.h5').replace('images','ground_truth'),'r')
groundtruth = np.asarray(gt_file['density'])
plt.imshow(groundtruth,cmap=CM.jet)

In [ ]:
np.sum(groundtruth)# don't mind this slight variation

In [ ]:
#now generate the ShanghaiB's ground truth
path_sets = [part_B_train,part_B_test]

In [ ]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [ ]:
for img_path in img_paths:
    print (img_path)
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    k = gaussian_filter(k,15)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
            hf['density'] = k